In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.gz.gz', compression='gzip')
df.head()

,tbrain_cc_training_48tags_hash_final.csv,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


In [3]:
# 更改欄位名稱
df.columns = ['dt'] + df.columns[1:].tolist()
df.head()

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


# 檢查缺失值

In [4]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
display(missing_values_table(df))

Your selected dataframe has 53 columns.
There are 53 columns that have missing values.


,Missing Values,% of Total Values
slam,1749907,5.3
age,421668,1.3
gender_code,421668,1.3
trdtp,436,0.0
educd,24,0.0
cuorg,11,0.0
poscd,11,0.0
naty,11,0.0
masts,11,0.0
card_4_txn_amt_pct,1,0.0


# 訓練測試切分

In [13]:
test_data = pd.read_csv('..\data\需預測的顧客名單及提交檔案範例.csv')
train_data = df.loc[set(df.index)-set(test_data.index)]
test_data = df.loc[test_data.index]
print(train_data.shape,test_data.shape)
display(train_data.head())
display(test_data.head())

(32475654, 53) (500000, 53)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
500000,1.0,10300515.0,45,3.0,13323.976180,3.0,0.0,0.0,0.0,1.0,...,1.0,3.0,12.0,1.0,2.0,30.0,146689.320675,1.0,4.0,1.0
500001,1.0,10049234.0,37,4.0,7628.067265,4.0,0.0,0.0,0.0,1.0,...,1.0,5.0,15.0,1.0,99.0,30.0,80355.055484,0.0,5.0,1.0
500002,1.0,10052786.0,5,1.0,5101.034666,0.0,0.0,0.0,1.0,0.0,...,2.0,3.0,2.0,1.0,99.0,30.0,33915.613854,1.0,4.0,1.0
500003,1.0,10098679.0,37,2.0,4756.920887,2.0,0.0,0.0,0.0,1.0,...,2.0,6.0,15.0,1.0,99.0,30.0,148279.373979,0.0,5.0,1.0
500004,1.0,10306817.0,25,1.0,5773.036485,1.0,0.0,0.0,0.0,1.0,...,1.0,2.0,8.0,1.0,2.0,30.0,95982.822967,0.0,4.0,1.0


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,1.00,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,1.00,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,0.00,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,0.75,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,1.00,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0


# 保存

In [17]:
train_data.reset_index().to_feather('../data/train_data.feather')
test_data.reset_index().to_feather('../data/test_data.feather')

# 讀取測試

In [20]:
train_df = pd.read_feather('../data/train_data.feather')
test_df = pd.read_feather('../data/test_data.feather')
display(train_df.head())
display(test_df.head())

,index,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,500000,1.0,10300515.0,45,3.0,13323.976180,3.0,0.0,0.0,0.0,...,1.0,3.0,12.0,1.0,2.0,30.0,146689.320675,1.0,4.0,1.0
1,500001,1.0,10049234.0,37,4.0,7628.067265,4.0,0.0,0.0,0.0,...,1.0,5.0,15.0,1.0,99.0,30.0,80355.055484,0.0,5.0,1.0
2,500002,1.0,10052786.0,5,1.0,5101.034666,0.0,0.0,0.0,1.0,...,2.0,3.0,2.0,1.0,99.0,30.0,33915.613854,1.0,4.0,1.0
3,500003,1.0,10098679.0,37,2.0,4756.920887,2.0,0.0,0.0,0.0,...,2.0,6.0,15.0,1.0,99.0,30.0,148279.373979,0.0,5.0,1.0
4,500004,1.0,10306817.0,25,1.0,5773.036485,1.0,0.0,0.0,0.0,...,1.0,2.0,8.0,1.0,2.0,30.0,95982.822967,0.0,4.0,1.0


,index,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,0,1.0,10321418.0,45,3.0,3891.965283,3.0,0.0,0.0,0.0,...,1.0,4.0,5.0,1.0,99.0,30.0,95982.822967,1.0,4.0,1.0
1,1,1.0,10414574.0,15,2.0,10616.561549,2.0,0.0,0.0,0.0,...,2.0,2.0,15.0,1.0,2.0,30.0,130702.351368,1.0,3.0,1.0
2,2,1.0,10134567.0,48,2.0,23527.655416,0.0,2.0,0.0,0.0,...,1.0,3.0,11.0,1.0,3.0,30.0,112010.611717,0.0,4.0,1.0
3,3,1.0,10001003.0,48,9.0,17751.558260,7.0,2.0,0.0,0.0,...,2.0,3.0,11.0,1.0,2.0,30.0,59701.507360,0.0,3.0,1.0
4,4,1.0,10267183.0,2,1.0,21701.307598,1.0,0.0,0.0,0.0,...,2.0,6.0,15.0,1.0,99.0,30.0,NaN,0.0,6.0,0.0
